In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [3]:
#BUHLMANN MODEL NON PARAMETRIZED ESTIMATES
def buhlmann_credibility(x):

    #CALCULATION OF EACH AVERAGE PER POLICY
    average_per_policy = np.zeros((x.shape[0], 1))

    for i in range(0, x.shape[0]):
        #average_per_policy[i, 0] = x[i,:].mean()
        average_per_policy[i, 0] = np.nanmean(x[i,:])

    #CALCULATION OF m 
    #average_global = average_per_policy.mean()
    average_global = np.nanmean(average_per_policy)

    #CALCULATION OF ELIGIBLE ELEMENTS FOR EACH POLICY
    elements_counter_vec = np.zeros(x.shape[0])

    for i in range(0, x.shape[0]):
        counter = 0
        for j in range(0, x.shape[1]):
            if np.isnan(x[i,j]) == False:
                counter += 1
        elements_counter_vec[i] = counter

    #CALCULATION OF ESTIMATOR SIGMA^2
    sigma_squared_hat = 0

    for i in range(0, x.shape[0]):
        for j in range(0, x.shape[1]):
            if np.isnan(x[i,j]) == False:
                sigma_squared_hat += 1/((elements_counter_vec - 1).sum())*(x[i, j] - average_per_policy[i, 0])**2

    #CALCULATION OF ESTIMATOR M^2
    M_squared_hat = 0 

    for i in range(0, x.shape[0]):
        M_squared_hat += 1/(x.shape[0] - 1)*(average_per_policy[i, 0] - average_global)**2
    
    M_squared_hat += -1/x.shape[1]*sigma_squared_hat

    #CALCULATION OF Z
    Z = (x.shape[1])/(x.shape[1] + sigma_squared_hat/M_squared_hat)

    #CALCULATION OF PREMIUM FOR EACH POLICY
    premium_per_policy = Z*average_per_policy + (1-Z)*average_global
    
    #OUTPUT

    print("----------------------")
    print("GLOBAL VARIABLES:")
    print("  Average Global (AKA. m) = " + str(average_global))
    print("  Sigma^2 = " + str(sigma_squared_hat))
    print("  M^2 = " + str(M_squared_hat))
    print("  K = " + str(sigma_squared_hat/M_squared_hat))
    print("  Z = " + str(Z))
    print("----------------------")
    print("POLICY DEPENDENT VARIABLES:")
    for i in range(0, x.shape[0]):
        print("  Policy " + str(i+1) + " Average = " + str(average_per_policy[i, 0]))
        print("  Policy " + str(i+1) + " Premium = " + str(premium_per_policy[i, 0]))
        print("  ----------------")

    return average_per_policy

In [4]:
# BUHLMANN STRAUB CREDIBILITY ESTIMATES: 

def buhlmann_straub_credibility(s,w):
    if s.shape != w.shape:
        print("Shapes of both inputs don't match, please check.")
        return
    
    #CREATE RATIO MATRIX:
    ratio_array = np.zeros(s.shape)

    for i in range(0, s.shape[0]):
        for j in range(0, s.shape[1]):
            if s[i,j] == np.NaN:
                ratio_array[i,j] = np.NaN
            else:
                ratio_array[i, j] = s[i,j]/w[i,j]

    #CALCULATE TOTAL WORKERS AND WEIGHTED AVERAGES FOR EACH POLICY:
    w_i = np.zeros((s.shape[0], 1))
    average_i = np.zeros((s.shape[0], 1))

    for i in range(0, s.shape[0]):
        average_i[i, 0] = np.nansum(s[i,:])/np.nansum(w[i,:])
        w_i[i, 0] = np.nansum(w[i,:])

    #CALCULATE GLOBAL AVERAGE m_hat
    m_hat = np.nansum(s)/np.nansum(w)

    #CALCULATION OF ELIGIBLE ELEMENTS FOR EACH POLICY
    elements_counter_vec = np.zeros(ratio_array.shape[0])

    for i in range(0, ratio_array.shape[0]):
        counter = 0
        for j in range(0, ratio_array.shape[1]):
            if np.isnan(ratio_array[i,j]) == False:
                counter += 1
        elements_counter_vec[i] = counter

    #CALCULATE SIGMA SQUARED 
    sigma_squared_hat = 0

    for i in range(0, s.shape[0]):
        for j in range(0, s.shape[1]):
            if np.isnan(ratio_array[i,j]) == False:
                sigma_squared_hat += 1/((elements_counter_vec - 1).sum())*w[i,j]*(ratio_array[i,j] - average_i[i, 0])**2

    #CALCULATE M SQUARED
    m_squared_hat = 0 

    for i in range(0, s.shape[0]):
        m_squared_hat += w_i.sum()/((w_i.sum())**2 - (w_i*w_i).sum())*w_i[i,0]*(average_i[i,0] - m_hat)**2

    m_squared_hat += -w_i.sum()/((w_i.sum())**2 - (w_i*w_i).sum())*(s.shape[0]-1)*sigma_squared_hat

    #CALCULATE K 
    K = sigma_squared_hat/m_squared_hat

    #CALCULATE Z FOR EACH POLICY
    Z_i = np.zeros((s.shape[0], 1))

    for i in range(0, s.shape[0]):
        Z_i[i,0] = w_i[i,0]/(w_i[i,0] + K)

    #CALCULATE EXPECTED LOSS FOR EACH POLICY
    Ex_Loss = np.zeros((s.shape[0], 1))

    for i in range(0, s.shape[0]):
        Ex_Loss[i,0] = Z_i[i,0]*average_i[i,0] + (1-Z_i[i,0])*m_hat
    
    #OUTPUT
    print("----------------------")
    print("GLOBAL VARIABLES:")
    print("  Average Global (AKA. m) = " + str(m_hat))
    print("  Sigma^2 = " + str(sigma_squared_hat))
    print("  M^2 = " + str(m_squared_hat))
    print("  K = " + str(K))
    print("----------------------")
    print("POLICY DEPENDENT VARIABLES:")
    for i in range(0, s.shape[0]):
        print("  Policy " + str(i+1) + " Average = " + str(average_i[i, 0]))
        print("  Policy " + str(i+1) + " Z Value = " + str(Z_i[i, 0]))
        print("  Policy " + str(i+1) + " Expected Loss = " + str(Ex_Loss[i, 0]))
        print("  ----------------")

    return

In [5]:
# Reserve: CHAIN LADDER METHOD

def reserving_chainladder(sq_mat, is_cumulative = True):

    #CHECK WHETHER IT IS CUMULATIVE OR NOT
    if is_cumulative == False:
        cumulative_matrix = np.zeros((sq_mat.shape[0], sq_mat.shape[1]))

        for i in range(0, cumulative_matrix.shape[0]):
            for j in range(0, cumulative_matrix.shape[1]-i):
                if j == 0:
                    cumulative_matrix[i,j] = sq_mat[i,j]
                else:
                    cumulative_matrix[i,j] = sq_mat[i,j] + cumulative_matrix[i,j-1]

    else:
        cumulative_matrix = sq_mat.astype(np.float64)

    #CALCULATE THE GROWTH RATIOS FOR EACH COL
    growth_vector = np.zeros(cumulative_matrix.shape[1])

    for i in range(1, cumulative_matrix.shape[1]):
        growth_vector[i] = cumulative_matrix[:(cumulative_matrix.shape[0] - i), i].sum()/cumulative_matrix[:(cumulative_matrix.shape[0] - i), i-1].sum()

    
    #CALCULATE THE NUMBERS FOR THE LOWER TRIANGLE OF THE MATRIX
    for i in range(1, cumulative_matrix.shape[1]):
        for j in range(cumulative_matrix.shape[0]-i, cumulative_matrix.shape[0]):
            cumulative_matrix[j, i] = round(growth_vector[i]*cumulative_matrix[j,i-1], 5)

    #CALCULATE LOSSES
    loss_vector = np.zeros(cumulative_matrix.shape[1])

    for i in range(0, cumulative_matrix.shape[0]):
        for j in range(cumulative_matrix.shape[1]-i-1, cumulative_matrix.shape[1]-i):
            loss_vector[i] = cumulative_matrix[i, cumulative_matrix.shape[1]-1] - cumulative_matrix[i, j]

    #CALCULATE INCREMENTAL AMOUNT
    incremental_matrix = np.zeros((sq_mat.shape[0], sq_mat.shape[1]))

    for i in range(0, incremental_matrix.shape[0]):
        for j in range(0, incremental_matrix.shape[1]):
            if j == 0:
                incremental_matrix[i,j] = cumulative_matrix[i,j]
            else:
                incremental_matrix[i,j] = cumulative_matrix[i,j] - cumulative_matrix[i,j-1]


    dash_string = "-----------------------------"
    #OUTPUT
    if is_cumulative == False:
        print("INCREMENTAL FINAL MATRIX:")
        print(incremental_matrix)
    print(dash_string)
    print("CUMULATIVE FINAL MATRIX:")
    print(cumulative_matrix)
    print(dash_string)
    print("YEARLY AND TOTAL RESERVES:")
    for i in range(0, len(loss_vector)):
        print("  Year " + str(i) + " Reserves = " + str(loss_vector[i]))
        print("  " + dash_string)
    print("  " + dash_string)
    print("  Total Reserves = " + str(loss_vector.sum()))
    
    return

In [6]:
#Reserve: LOSS RATIO METHOD

def reserving_lossratio(perc, premiums, paid):
    print("Total Earned Premium = " + str(premiums.sum()))
    print("Total Paid Amount = " + str(paid.sum()))
    print("Total Reserve Amount = " + str(perc*premium.sum() - paid.sum()))

In [7]:
# Reserve: BORNHUETTER-FERGUSON METHOD

def reserving_bornhuetter(sq_mat, premium_vector, loss_ratio, is_cumulative = True):

    #CHECK WHETHER IT IS CUMULATIVE OR NOT
    if is_cumulative == False:
        cumulative_matrix = np.zeros((sq_mat.shape[0], sq_mat.shape[1]))

        for i in range(0, cumulative_matrix.shape[0]):
            for j in range(0, cumulative_matrix.shape[1]-i):
                if j == 0:
                    cumulative_matrix[i,j] = sq_mat[i,j]
                else:
                    cumulative_matrix[i,j] = sq_mat[i,j] + cumulative_matrix[i,j-1]

    else:
        cumulative_matrix = sq_mat.astype(np.float64)


    #CALCULATE THE GROWTH RATIOS FOR EACH COL
    growth_vector = np.zeros(cumulative_matrix.shape[1]) + 1

    for i in range(1, cumulative_matrix.shape[1]):
        growth_vector[i] = cumulative_matrix[:(cumulative_matrix.shape[0] - i), i].sum()/cumulative_matrix[:(cumulative_matrix.shape[0] - i), i-1].sum()

    
    #CALCULATE THE NUMBERS FOR THE LOWER TRIANGLE OF THE MATRIX
    for i in range(1, cumulative_matrix.shape[1]):
        for j in range(cumulative_matrix.shape[0]-i, cumulative_matrix.shape[0]):
            cumulative_matrix[j, i] = round(growth_vector[i]*cumulative_matrix[j,i-1], 5)


    #CALCULATE DEVELOPMENT FACTORS:
    development_factor = np.zeros(cumulative_matrix.shape[1]) + 1

    for i in range(1, len(development_factor)):
        development_factor[i] = np.prod(growth_vector[-i:])


    #CALCULATE LOSSES
    loss_vector = np.zeros(cumulative_matrix.shape[1])

    for i in range(0, len(loss_vector)):
        loss_vector[i] = premium_vector[i]*loss_ratio*(1-1/development_factor[i])

    #CALCULATE INCREMENTAL AMOUNT
    incremental_matrix = np.zeros((sq_mat.shape[0], sq_mat.shape[1]))

    for i in range(0, incremental_matrix.shape[0]):
        for j in range(0, incremental_matrix.shape[1]):
            if j == 0:
                incremental_matrix[i,j] = cumulative_matrix[i,j]
            else:
                incremental_matrix[i,j] = cumulative_matrix[i,j] - cumulative_matrix[i,j-1]


    dash_string = "-----------------------------"
    #OUTPUT
    if is_cumulative == False:
        print("INCREMENTAL FINAL MATRIX:")
        print(incremental_matrix)
    print(dash_string)
    print("CUMULATIVE FINAL MATRIX:")
    print(cumulative_matrix)
    print(dash_string)
    print("YEARLY AND TOTAL RESERVES:")
    for i in range(0, len(loss_vector)):
        print("  Year " + str(i) + " Premium*Loss Ratio = " + str(premium_vector[i]*loss_ratio))
        print("  Year " + str(i) + " Developement Factor = " + str(development_factor[i]))
        print("  Year " + str(i) + " Reserves = " + str(loss_vector[i]))
        print("  " + dash_string)
    print("  " + dash_string)
    print("  Total Reserves = " + str(loss_vector.sum()))


    return 

In [8]:
####################################################
############# ENTER YOUR INPUTS BELOW ##############
####################################################

In [9]:
#TESTING PURPOSES EXAMPLES


'''
#################TEST 1######################
experience = np.array([[1,3,2,5,4],
                       [2,0,1,2,0],
                       [3,2,2,1,2]
                      ])

buhlmann_credibility(experience)

#################TEST 2######################
experience = np.array([[0,1,2,1,2,0],
                       [3,4,2,1,4,4],
                       [3,3,2,1,2,1]
                      ])

buhlmann_credibility(experience)

#################TEST 3######################
experience = np.array([[4,3,5,0,0,1],
                       [1,2,3,2,1,2],
                       [4,4,4,3,5,1]
                      ])

buhlmann_credibility(experience)

#################TEST 4######################
s = np.array([[1,1,3,2],
              [7,5,7,7],
              [11,11,13,12]
             ])

w = np.array([[1,3,2,2],
              [1,1,2,2],
              [2,2,2,2]
             ])

buhlmann_straub_credibility(s,w)

#################TEST 5######################
s = np.array([[0,1,0,4,0,1],
              [0,1,0,4,0,1]
             ])

w = np.array([[0.15,0.175,0.25,0.25,0.1,0.122],
              [0.275,0.35,0.35,0.2,0.222,0.23]
             ])

buhlmann_straub_credibility(s,w)


#################TEST 6######################
sq_mat = np.array([[786, 1410, 2216, 2440, 2519],
                    [904, 1575, 2515, 2795, 0],
                    [995, 1814, 2880, 0, 0],
                    [1220, 2142, 0, 0 ,0],
                    [1182, 0, 0, 0, 0]
                   ])

reserving_chainladder(sq_mat, is_cumulative = True)


#################TEST 7######################
sq_mat = np.array([ [26312,31467,24672,13055,6158],
                    [30470,35012,25491,12589,0],
                    [49756,51831,35267,0,0],
                    [50420,52315, 0, 0 ,0],
                    [56762, 0, 0, 0, 0]
                   ])

reserving_chainladder(sq_mat, is_cumulative = False)


#################TEST 8######################
loss_ratio = 0.60
premium = np.array([100000, 105000, 110000, 112000, 120000, 115000])
paid = np.array([58000, 50000, 45000, 40000, 25000, 12000])

reserving_lossratio(loss_ratio, premiums, paid)


#################TEST 9######################
sq_mat = np.array([[786, 1410, 2216, 2440, 2519],
                    [904, 1575, 2515, 2795, 0],
                    [995, 1814, 2880, 0, 0],
                    [1220, 2142, 0, 0 ,0],
                    [1182, 0, 0, 0, 0]
                   ])

premium_vector = np.array([2908, 3364, 3801, 4564, 4443])

reserving_bornhuetter(sq_mat, premium_vector, 0.86)

#################TEST 10######################
s = np.array([[3,2,2,0],
              [2,1,0,np.NaN]
             ])

w = np.array([[2,2,2,1],
              [4,3,2,np.NaN]
             ])

buhlmann_straub_credibility(s,w)

'''

'\n#################TEST 1######################\nexperience = np.array([[1,3,2,5,4],\n                       [2,0,1,2,0],\n                       [3,2,2,1,2]\n                      ])\n\nbuhlmann_credibility(experience)\n\n#################TEST 2######################\nexperience = np.array([[0,1,2,1,2,0],\n                       [3,4,2,1,4,4],\n                       [3,3,2,1,2,1]\n                      ])\n\nbuhlmann_credibility(experience)\n\n#################TEST 3######################\nexperience = np.array([[4,3,5,0,0,1],\n                       [1,2,3,2,1,2],\n                       [4,4,4,3,5,1]\n                      ])\n\nbuhlmann_credibility(experience)\n\n#################TEST 4######################\ns = np.array([[1,1,3,2],\n              [7,5,7,7],\n              [11,11,13,12]\n             ])\n\nw = np.array([[1,3,2,2],\n              [1,1,2,2],\n              [2,2,2,2]\n             ])\n\nbuhlmann_straub_credibility(s,w)\n\n#################TEST 5####################

In [12]:
#################TEST 4######################
s = np.array([[3,1,0,2],
              [7,6,7,8],
              [12,13,12,np.NaN]
             ])

w = np.array([[2,3,2,1],
              [3,2,1,2],
              [2,1,3,np.NaN]
             ])

buhlmann_straub_credibility(s,w)

----------------------
GLOBAL VARIABLES:
  Average Global (AKA. m) = 3.227272727272727
  Sigma^2 = 10.3125
  M^2 = 5.562239583333335
  K = 1.8540193829299119
----------------------
POLICY DEPENDENT VARIABLES:
  Policy 1 Average = 0.75
  Policy 1 Z Value = 0.8118514576760804
  Policy 1 Expected Loss = 1.2160952525751645
  ----------------
  Policy 2 Average = 3.5
  Policy 2 Z Value = 0.8118514576760804
  Policy 2 Expected Loss = 3.4486867611843857
  ----------------
  Policy 3 Average = 6.166666666666667
  Policy 3 Z Value = 0.7639400550806539
  Policy 3 Expected Loss = 5.472793495237074
  ----------------
